In [1]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import cv2
from tqdm import tqdm
import multiprocessing as mp
from multiprocessing import Pool
tqdm.pandas(desc="my bar!")
import os
import datetime
print(os.listdir("Birds"))

['test', 'test1', 'train']


C:\Users\april\Anaconda3\lib\site-packages\tqdm\std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


#Full training set: CUB dataset 40 classes+Grocery dataset 20 classes.

#Full test set: CUB 14 classes+Grocery 11 classes
#Train and test sets are totally disjoint.


In [2]:
train_dir = os.listdir('Birds/train/') 
datax = np.array([])

### CUB-Birds: Train clsses[1-40], test class[170-183], NO AUGMENTATION, STACK ONLY 1 TIME


In [3]:
charct = os.listdir('Birds/train/')
alphabet_directory_train='Birds/train/'
trdatay = []
trdatax=[]
for character in charct:
        images = os.listdir('Birds/train/' + character + '/')
        for img in images:
            image = cv2.resize(cv2.imread(alphabet_directory_train + character + '/' + img), (42,42))
            #image = np.expand_dims(image, 0)
            trdatax.append( image)
            trdatay.append(character)

In [4]:
trainx=np.array(trdatax)
trainy=np.array(trdatay)
trainx.shape, trainy.shape

((2290, 42, 42, 3), (2290,))

In [5]:
trainx

array([[[[176, 177, 173],
         [171, 172, 168],
         [160, 160, 154],
         ...,
         [184, 184, 178],
         [179, 181, 175],
         [184, 186, 180]],

        [[157, 158, 154],
         [160, 161, 157],
         [166, 167, 162],
         ...,
         [183, 183, 177],
         [185, 187, 182],
         [188, 190, 184]],

        [[174, 175, 171],
         [182, 183, 179],
         [180, 181, 178],
         ...,
         [187, 187, 181],
         [177, 179, 173],
         [167, 169, 163]],

        ...,

        [[162, 163, 159],
         [122, 125, 116],
         [126, 130, 120],
         ...,
         [162, 163, 158],
         [161, 166, 157],
         [150, 155, 147]],

        [[163, 164, 159],
         [163, 165, 159],
         [167, 170, 163],
         ...,
         [161, 163, 157],
         [175, 178, 176],
         [171, 174, 172]],

        [[162, 163, 158],
         [167, 169, 166],
         [178, 179, 177],
         ...,
         [160, 162, 156],
        

In [6]:
trainy

array(['001.Black_footed_Albatross', '001.Black_footed_Albatross',
       '001.Black_footed_Albatross', ..., '040.Olive_sided_Flycatcher',
       '040.Olive_sided_Flycatcher', '040.Olive_sided_Flycatcher'],
      dtype='<U28')

In [39]:
characters = os.listdir('Birds/test/')
alphabet_directory_test='Birds/test/'
datay = []
datax=[]
for character in characters:
        images = os.listdir('Birds/test/' + character + '/')
        for img in images:
            image = cv2.resize(cv2.imread(alphabet_directory_test + character + '/' + img), (42,42))
            #image = np.expand_dims(image, 0)
            datax.append( image)
            datay.append(character)

In [40]:
testx=np.array(datax)

In [41]:
testx.shape

(834, 42, 42, 3)

In [42]:
testy=np.array(datay)


In [43]:
testy.shape

(834,)

# Add Grocery set, use original rotation 4 times as augmentation

In [28]:
train_Grocery = os.listdir('Grocery/train/') #juice / milk /oatghurt / oat-milk / sour-cream
datax = np.array([])
def image_rotate(img, angle):
    """
    Image rotation at certain angle. It is used for data augmentation 
    """
    rows,cols, _ = img.shape
    M = cv2.getRotationMatrix2D((cols/2 ,rows/2),angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    return np.expand_dims(dst, 0)

def read_alphabets(alphabet_directory, directory):
    """
    Reads all the characters from alphabet_directory and augment each image with 90, 180, 270 degrees of rotation.
    """
    datax = None
    datay = []
    characters = os.listdir(alphabet_directory)
    for character in characters:
        images = os.listdir(alphabet_directory + character + '/')
        for img in images:
            image = cv2.resize(cv2.imread(alphabet_directory + character + '/' + img), (42,42))
            image90 = image_rotate(image, 90)
            image180 = image_rotate(image, 180)
            image270 = image_rotate(image, 270)
            image = np.expand_dims(image, 0)
            if datax is None:
                datax = np.vstack([image, image90, image180, image270])
            else:
                datax = np.vstack([datax, image, image90, image180, image270])
            datay.append(directory + '_' + character + '_0')
            datay.append(directory + '_' + character + '_90')
            datay.append(directory + '_' + character + '_180')
            datay.append(directory + '_' + character + '_270')
    return datax, np.array(datay)

def read_images(base_directory):
    """
    Used multithreading for data reading to decrease the reading time drastically
    """
    datax = None
    datay = []
    
    results = [read_alphabets(base_directory + '/' + directory + '/', directory, ) for directory in os.listdir(base_directory)]
  
    for result in results:
        if datax is None:
            datax = result[0]
            datay = result[1]
        else:
            datax = np.vstack([datax, result[0]])
            datay = np.concatenate([datay, result[1]])
    return datax, datay

In [29]:
%time G_trainx, G_trainy = read_images('Grocery/train/Packages/')


Wall time: 1.13 s


In [30]:
G_trainx.shape

(2168, 42, 42, 3)

In [31]:
G_trainy

array(['Juice_Bravo-Apple-Juice_0', 'Juice_Bravo-Apple-Juice_90',
       'Juice_Bravo-Apple-Juice_180', ...,
       'Sour-Cream_Arla-Sour-Cream_90', 'Sour-Cream_Arla-Sour-Cream_180',
       'Sour-Cream_Arla-Sour-Cream_270'], dtype='<U48')

In [32]:
%time G_testx, G_testy = read_images('Grocery/test/Packages/')

Wall time: 596 ms


In [33]:
G_testx.shape,G_testy.shape

((1288, 42, 42, 3), (1288,))

In [34]:
G_testy

array(['Sour-Milk_Arla-Sour-Milk_0', 'Sour-Milk_Arla-Sour-Milk_90',
       'Sour-Milk_Arla-Sour-Milk_180', ...,
       'Yoghurt_Yoggi-Vanilla-Yoghurt_90',
       'Yoghurt_Yoggi-Vanilla-Yoghurt_180',
       'Yoghurt_Yoggi-Vanilla-Yoghurt_270'], dtype='<U45')

In [35]:
Full_testy=np.hstack([testy, G_testy])

In [36]:
Full_testy

array(['170.Mourning_Warbler', '170.Mourning_Warbler',
       '170.Mourning_Warbler', ..., 'Yoghurt_Yoggi-Vanilla-Yoghurt_90',
       'Yoghurt_Yoggi-Vanilla-Yoghurt_180',
       'Yoghurt_Yoggi-Vanilla-Yoghurt_270'], dtype='<U45')

In [37]:
Full_testy.shape

(2122,)

In [44]:
Full_testx=np.vstack([testx, G_testx])
Full_testx

array([[[[ 64, 102,  92],
         [ 37,  74,  65],
         [ 57, 107,  95],
         ...,
         [ 15,  44,  29],
         [ 19,  48,  33],
         [ 27,  62,  46]],

        [[ 44,  89,  83],
         [ 49, 100,  83],
         [ 57,  99,  90],
         ...,
         [ 16,  41,  31],
         [ 17,  44,  34],
         [ 73,  92,  69]],

        [[ 81, 136, 127],
         [ 37,  83,  73],
         [ 57, 104,  96],
         ...,
         [ 11,  42,  27],
         [ 14,  48,  34],
         [ 56,  83,  65]],

        ...,

        [[  8,  57,  47],
         [ 19,  82,  71],
         [ 40, 117, 103],
         ...,
         [ 12,  29,  26],
         [ 19,  24,  24],
         [  5,  10,   9]],

        [[  8,  46,  34],
         [ 18,  78,  65],
         [ 36, 114, 101],
         ...,
         [  2,  44,  31],
         [ 15,  39,  35],
         [ 10,  16,  14]],

        [[  3,  44,  30],
         [ 19,  74,  62],
         [ 65, 125, 119],
         ...,
         [111, 174, 182],
        

In [45]:
Full_testx.shape

(2122, 42, 42, 3)

# # stack up: birds train+Grocery train

In [47]:
Full_trainx=np.vstack([trainx, G_trainx])
Full_trainy=np.hstack([trainy, G_trainy])

In [48]:
Full_trainx.shape, Full_trainx.shape

((4458, 42, 42, 3), (4458, 42, 42, 3))

### classes for train , class[1-100], test classes [170-200]

In [46]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tqdm import trange
from time import sleep
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
use_gpu = torch.cuda.is_available()

In [49]:
Full_trainx = torch.from_numpy(Full_trainx).float()
Full_testx = torch.from_numpy(Full_testx).float()

In [ ]:
#To avoid CUDA out of memory error
    #Send the batches to CUDA iteratively, and make small batch sizes.
    #Don't send all your data to CUDA at once in the beginning. 
"""
if use_gpu:
    trainx = trainx.cuda()
    testx = testx.cuda()#.half() #
trainx.size()""" #, testx.size()
# use dtypes that use less memory. For instance, torch.float16 or torch.half.

In [51]:
Full_trainx = Full_trainx.permute(0,3,1,2)
Full_testx = Full_testx.permute(0,3,1,2)

# Image2Vector CNN (we can use other pre-trained model )

In [53]:
class Net(nn.Module):
    """
    Image2Vector CNN which takes image of dimension (42x42x3) and return column vector length 224
    """
    def sub_block(self, in_channels, out_channels=64, kernel_size=3):
        block = torch.nn.Sequential(
                    torch.nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=out_channels, padding=1),
                    torch.nn.BatchNorm2d(out_channels),
                    torch.nn.ReLU(),
                    torch.nn.MaxPool2d(kernel_size=2)
                )
        return block
    
    def __init__(self):
        super(Net, self).__init__()
        self.convnet1 = self.sub_block(3)
        self.convnet2 = self.sub_block(64)
        self.convnet3 = self.sub_block(64)
        self.convnet4 = self.sub_block(64)

    def forward(self, x):
        x = self.convnet1(x)
        x = self.convnet2(x)
        x = self.convnet3(x)
        x = self.convnet4(x)
        x = torch.flatten(x, start_dim=1)
        return x

# PrototypicalNet

In [54]:
class PrototypicalNet(nn.Module):
    def __init__(self, use_gpu=False):
        super(PrototypicalNet, self).__init__()       
        self.f = Net()
        #self.f =model
        self.gpu = use_gpu
        if self.gpu:
            self.f = self.f.cuda()
    
    def forward(self, datax, datay, Ns,Nc, Nq, total_classes):
        """
        Implementation of one episode in Prototypical Net
        datax: Training images
        datay: Corresponding labels of datax
        Nc: Number  of classes per episode
        Ns: Number of support data per class
        Nq:  Number of query data per class
        total_classes: Total classes in training set
        """
        k = total_classes.shape[0]
        K = np.random.choice(total_classes, Nc, replace=False)
        Query_x = torch.Tensor()
        if(self.gpu):
            Query_x = Query_x.cuda()
        Query_y = []
        Query_y_count = []
        centroid_per_class  = {}
        class_label = {}
        label_encoding = 0
        for cls in K:
            S_cls, Q_cls = self.random_sample_cls(datax, datay, Ns, Nq, cls)
            centroid_per_class[cls] = self.get_centroid(S_cls, Ns)
            class_label[cls] = label_encoding
            label_encoding += 1
            Query_x = torch.cat((Query_x, Q_cls), 0) # Joining all the query set together
            Query_y += [cls]
            Query_y_count += [Q_cls.shape[0]]
        Query_y, Query_y_labels = self.get_query_y(Query_y, Query_y_count, class_label)
        Query_x = self.get_query_x(Query_x, centroid_per_class, Query_y_labels)
        return Query_x, Query_y
    
    def random_sample_cls(self, datax, datay, Ns, Nq, cls):
        """
        Randomly samples Ns examples as support set and Nq as Query set
        """
        data = datax[(datay == cls).nonzero()]
        perm = torch.randperm(data.shape[0])
        idx = perm[:Ns]
        S_cls = data[idx]
        idx = perm[Ns : Ns+Nq]
        Q_cls = data[idx]
        if self.gpu:
            S_cls = S_cls.cuda() 
            Q_cls = Q_cls.cuda() 
        return S_cls, Q_cls
    
    def get_centroid(self, S_cls, Nc):
        """
        Returns a centroid vector of support set for a class
        """
        return torch.sum(self.f(S_cls), 0).unsqueeze(1).transpose(0,1) / Nc
    
    def get_query_y(self, Qy, Qyc, class_label):
        """
        Returns labeled representation of classes of Query set and a list of labels.
        """
        labels = []
        m = len(Qy)
        for i in range(m):
            labels += [Qy[i]] * Qyc[i]
        labels = np.array(labels).reshape(len(labels), 1)
        label_encoder = LabelEncoder()
        Query_y = torch.Tensor(label_encoder.fit_transform(labels).astype(int)).long()
        if self.gpu:
            Query_y = Query_y.cuda()
        Query_y_labels = np.unique(labels)
        return Query_y, Query_y_labels
    
    def get_centroid_matrix(self, centroid_per_class, Query_y_labels):
        """
        Returns the centroid matrix where each column is a centroid of a class.
        """
        centroid_matrix = torch.Tensor()
        if(self.gpu):
            centroid_matrix = centroid_matrix.cuda()
        for label in Query_y_labels:
            centroid_matrix = torch.cat((centroid_matrix, centroid_per_class[label]))
        if self.gpu:
            centroid_matrix = centroid_matrix.cuda()
        return centroid_matrix
    
    def get_query_x(self, Query_x, centroid_per_class, Query_y_labels):
        """
        Returns distance matrix from each Query image to each centroid.
        """
        centroid_matrix = self.get_centroid_matrix(centroid_per_class, Query_y_labels)
        Query_x = self.f(Query_x)
        m = Query_x.size(0)
        n = centroid_matrix.size(0)
        # The below expressions expand both the matrices such that they become compatible to each other in order to caclulate L2 distance.
        centroid_matrix = centroid_matrix.expand(m, centroid_matrix.size(0), centroid_matrix.size(1)) # Expanding centroid matrix to "m".
        Query_matrix = Query_x.expand(n, Query_x.size(0), Query_x.size(1)).transpose(0,1) # Expanding Query matrix "n" times
        Qx = torch.pairwise_distance(centroid_matrix.transpose(1,2), Query_matrix.transpose(1,2))
        return Qx

In [55]:
protonet = PrototypicalNet(use_gpu=use_gpu)
optimizer = optim.SGD(protonet.parameters(), lr = 0.01, momentum=0.99)

# Training

In [56]:
def train_step(datax, datay, Ns,Nc, Nq):
    optimizer.zero_grad()
    Qx, Qy= protonet(datax, datay, Ns, Nc, Nq, np.unique(datay))
    pred = torch.log_softmax(Qx, dim=-1)
    loss = F.nll_loss(pred, Qy)
    loss.backward()
    optimizer.step()
    acc = torch.mean((torch.argmax(pred, 1) == Qy).float())
    return loss, acc

In [57]:
num_episode = 20000
frame_size = 1000

In [58]:
frame_loss = 0
frame_acc = 0
for i in range(num_episode):
    #To avoid CUDA out of memory error
    #Send the batches to CUDA iteratively, and make small batch sizes.
    #Don't send all your data to CUDA at once in the beginning. Rather, do it as follows: 
    if use_gpu:
        Full_trainx = Full_trainx.cuda() 
     
    loss, acc = train_step(Full_trainx, Full_trainy, 5, 20, 15) # is Nc: Number  of classes per episode, can by tuned later
    frame_loss += loss.data
    frame_acc += acc.data
    if( (i+1) % frame_size == 0):
        print(datetime.datetime.now(),"Frame Number:", ((i+1) // frame_size), 'Frame Loss: ', frame_loss.data.cpu().numpy().tolist()/ frame_size, 'Frame Accuracy:', (frame_acc.data.cpu().numpy().tolist() * 100) / frame_size)
        frame_loss = 0
        frame_acc = 0

C:\Users\april\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2020-05-03 09:46:33.563066 Frame Number: 1 Frame Loss:  1.88402294921875 Frame Accuracy: 31.610336303710938
2020-05-03 09:48:45.728083 Frame Number: 2 Frame Loss:  1.10334912109375 Frame Accuracy: 58.553717041015624
2020-05-03 09:50:58.021913 Frame Number: 3 Frame Loss:  0.8382232666015625 Frame Accuracy: 68.29754028320312
2020-05-03 09:53:10.208979 Frame Number: 4 Frame Loss:  0.7029210815429687 Frame Accuracy: 73.492431640625
2020-05-03 09:55:22.447417 Frame Number: 5 Frame Loss:  0.6033626708984375 Frame Accuracy: 77.1947998046875
2020-05-03 09:57:33.018901 Frame Number: 6 Frame Loss:  0.5365086669921875 Frame Accuracy: 79.66160888671875
2020-05-03 09:59:43.615490 Frame Number: 7 Frame Loss:  0.4714530944824219 Frame Accuracy: 82.08367309570312
2020-05-03 10:01:54.170033 Frame Number: 8 Frame Loss:  0.4051081848144531 Frame Accuracy: 84.55684814453124
2020-05-03 10:04:04.757168 Frame Number: 9 Frame Loss:  0.37212741088867185 Frame Accuracy: 85.66818237304688
2020-05-03 10:06:15.512

# Testing

In [59]:
def test_step(datax, datay, Ns,Nc, Nq):
    Qx, Qy= protonet(datax, datay, Ns, Nc, Nq, np.unique(datay))
    pred = torch.log_softmax(Qx, dim=-1)
    loss = F.nll_loss(pred, Qy)
    acc = torch.mean((torch.argmax(pred, 1) == Qy).float())
    return loss, acc

In [60]:
num_test_episode = 5000
avg_loss = 0
avg_acc = 0
for _ in range(num_test_episode):
    if use_gpu:
        Full_testx = Full_testx.cuda()
    loss, acc = test_step(Full_testx, Full_testy, 5, 5, 15)
    avg_loss += loss.data
    avg_acc += acc.data
print('Avg Loss: ', avg_loss.data.cpu().numpy().tolist() / num_test_episode , 'Avg Accuracy:', (avg_acc.data.cpu().numpy().tolist() * 100) / num_test_episode)


Avg Loss:  1.0831509765625 Avg Accuracy: 71.2436767578125
